In [ ]:
# transformer with phi3 is only available in latest version
!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fqv1ehnl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fqv1ehnl
  Resolved https://github.com/huggingface/transformers to commit 96eb06286b63c9c93334d507e632c175d6ba8b28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9140346 sha256=8aa7ed717b4cc4bb17ead4606b73ce84712e98e6a8751b4de89a0bfc1c9e1368
  Stored in directory: /tmp/pip-ephem-wheel-cache-vfz7_y0m/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
!pip install "pandas<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


##  Import Packages

In [ ]:
import pandas
import torch
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset, load_dataset, load_from_disk

In [ ]:
# Setting torch seed
torch.random.manual_seed(0)

## Defining model & tokenizers

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Phi-3 Demo

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user.",
    },
    {
        "role": "user",
        "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
    },
    {
        "role": "assistant",
        "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.",
    },
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 To solve the equation 2x + 3 = 7, you need to isolate the variable x. Here are the steps:

1. Subtract 3 from both sides of the equation to get rid of the +3 on the left side. This gives you: 2x = 7 - 3, which simplifies to 2x = 4.

2. Now, divide both sides of the equation by 2 to solve for x. This gives you: x = 4 / 2, which simplifies to x = 2.

So, the solution to the equation 2x + 3 = 7 is x = 2.


## Using Phi-3 for ABSA

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]

In [ ]:
messages = [
    {
        "role": "system",
        "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
    },
    {
        "role": "user",
        "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
    },
    {
        "role": "assistant",
        "content": "{'service':'positive','food':'positive','price':'positive'}",
    },
    {
        "role": "user",
        "content": "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area.",
    },
]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)

In [ ]:
print(output[0]["generated_text"])

 {'food':'negative','price':'neutral'}


## Loading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading test data
dataset = load_from_disk("/content/drive/MyDrive/ds/test_dataset.hf")
dataset[0]

{'text': 'bread top notch well',
 'category': ['food'],
 'polarity': ['positive'],
 'joint': ['food#positive'],
 'category_labels': [0, 1, 0, 0, 0],
 'polarity_labels': [0, 0, 1],
 'joint_labels': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'true_labels': {'ambience': None,
  'food': 'positive',
  'other': None,
  'price': None,
  'service': None}}

## Zero shot

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


def zero_shot_classify(data):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
                },
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]
    try:
        _output = [eval(o) for o in output]
    except Exception as e:
        print(f"Output not a valid json. \nException: {e} \nOutput: {output}")
        print("\n")
        _output = [{} for o in output]

    return {"zero_shot_result": _output}

In [ ]:
"""messages = [
    {"role": "system", "content": f'Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.'},
    {"role": "user", "content": 'Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening'},
    ]"""

'messages = [\n    {"role": "system", "content": f\'Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.\'},\n    {"role": "user", "content": \'Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening\'},\n    ]'

In [ ]:
dataset_zs = dataset.map(
    lambda _rows: zero_shot_classify(_rows), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
dataset_zs.to_pandas().to_csv(
    "/content/drive/MyDrive/results/phi3_zero_shot_df.csv"
)

### One shot

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


def one_shot_classify(data):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
                },
                {
                    "role": "user",
                    "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
                },
                {
                    "role": "assistant",
                    "content": "{'service':'positive','food':'positive','price':'positive'}",
                },
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]

    #   for o in output:
    #       try:
    #           _output = eval(o.strip())
    #           parsed_output.append(_output)
    #       except Exception as e:
    #           print(f"Output not a valid json. \nException: {e} \nOutput: {o}")
    #           parsed_output.append({})
    #       print("\n")
    return {"one_shot_result": output}

In [ ]:
dataset_os = dataset.map(
    lambda _rows: one_shot_classify(_rows), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
dataset_os.to_pandas().to_csv(
    "/content/drive/MyDrive/results/phi3_one_shot_df.csv"
)

## Two Shot

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


def two_shot_classify(data):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
                },
                {
                    "role": "user",
                    "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
                },
                {
                    "role": "assistant",
                    "content": "{'service':'positive','food':'positive','price':'positive'}",
                },
                {
                    "role": "user",
                    "content": "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area.",
                },
                {
                    "role": "assistant",
                    "content": "{'price':'negative','food':'negative'}",
                },
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]

    #   for o in output:
    #       try:
    #           _output = eval(o.strip())
    #           parsed_output.append(_output)
    #       except Exception as e:
    #           print(f"Output not a valid json. \nException: {e} \nOutput: {o}")
    #           parsed_output.append({})
    #       print("\n")
    return {"two_shot_result": output}

In [ ]:
dataset_ts = dataset.map(
    lambda _rows: two_shot_classify(_rows), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
dataset_ts.to_pandas().to_csv(
    "/content/drive/MyDrive/results/phi3_two_shot_df.csv"
)

## Three shot

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


def three_shot_classify(data):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
                },
                {
                    "role": "user",
                    "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
                },
                {
                    "role": "assistant",
                    "content": "{'service':'positive','food':'positive','price':'positive'}",
                },
                {
                    "role": "user",
                    "content": "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area.",
                },
                {
                    "role": "assistant",
                    "content": "{'price':'negative','food':'negative'}",
                },
                {
                    "role": "user",
                    "content": "I would definitely go back -- if only for some of those exotic martinis on the blackboard.",
                },
                {"role": "assistant", "content": "{'food':'positive'}"},
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]

    #   for o in output:
    #       try:
    #           _output = eval(o.strip())
    #           parsed_output.append(_output)
    #       except Exception as e:
    #           print(f"Output not a valid json. \nException: {e} \nOutput: {o}")
    #           parsed_output.append({})
    #       print("\n")
    return {"three_shot_result": output}

In [ ]:
dataset_3s = dataset.map(
    lambda _rows: three_shot_classify(_rows), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:

dataset_3s.to_pandas().to_csv(
    "/content/drive/MyDrive/results/phi3_three_shot_df.csv"
)

## Five shot

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


def five_shot_classify(data):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
                },
                {
                    "role": "user",
                    "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
                },
                {
                    "role": "assistant",
                    "content": "{'service':'positive','food':'positive','price':'positive'}",
                },
                {
                    "role": "user",
                    "content": "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area.",
                },
                {
                    "role": "assistant",
                    "content": "{'price':'negative','food':'negative'}",
                },
                {
                    "role": "user",
                    "content": "I would definitely go back -- if only for some of those exotic martinis on the blackboard.",
                },
                {"role": "assistant", "content": "{'food':'positive'}"},
                {
                    "role": "user",
                    "content": "Definitely a great spot for a nice occasion or date.",
                },
                {
                    "role": "assistant",
                    "content": "{'other':'neutral'}",
                },
                {
                    "role": "user",
                    "content": "The food was terrific and the service classy, attentive, without being overbearing.",
                },
                {
                    "role": "assistant",
                    "content": "{'food':'positive','service':'positive'}",
                },
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]

    #   for o in output:
    #       try:
    #           _output = eval(o.strip())
    #           parsed_output.append(_output)
    #       except Exception as e:
    #           print(f"Output not a valid json. \nException: {e} \nOutput: {o}")
    #           parsed_output.append({})
    #       print("\n")
    return {"five_shot_result": output}

In [ ]:
dataset_5s = dataset.map(
    lambda _rows: five_shot_classify(_rows), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:

dataset_5s.to_pandas().to_csv(
    "/content/drive/MyDrive/results/phi3_five_shot_df.csv"
)